# Lab 2 INPUT FROM SWITCHES TO TURN ON CURRESPONSING LED'S

In [ ]:
import time
import RPi.GPIO as gpio # type: ignore

gpio.setwarnings(False)
gpio.setmode(gpio.BOARD)

led1 = 15
led2 = 13

switch1 = 37    
switch2 = 35

gpio.setup(led1,gpio.OUT,initial=0)
gpio.setup(led2,gpio.OUT,initial=0)
gpio.setup(switch1,gpio.IN)
gpio.setup(switch2,gpio.IN)

def glow_led(event):
    if event == switch1:
        gpio.output(led1,True)
        time.sleep(3)
        gpio.output(led1,False)
        
    elif event == switch2:
        gpio.output(led2,True)
        time.sleep(3)
        gpio.output(led2,False)
    
gpio.add_event_detect(switch1,gpio.RISING,callback=glow_led,bouncetime=1)
gpio.add_event_detect(switch2,gpio.RISING,callback=glow_led,bouncetime=1)

try:
    while(True):
        time.sleep(1)
        
except KeyboardInterrupt:
    gpio.cleanup()

# Lab 3 FLASH AN LED ON GIVEN ON_TIME AND OFF_TIME CYCLE. WHERE THE TWO TIMES ARE TAKEN FROM THE FILE.


In [ ]:
import time
import RPi.GPIO as gpio

gpio.setwarnings(False)
gpio.setmode(gpio.BOARD)

led = 15
gpio.setup(led,gpio.OUT,initial=0)

file = open('Lab3.txt','r')
lines = file.readlines()

ON_TIME = int(lines[0].split("=")[1])
OFF_TIME = int(lines[1].split("=")[1])

try:
    while(True):
        gpio.output(led,True)
        time.sleep(ON_TIME)
        gpio.output(led,False)
        time.sleep(OFF_TIME)
except KeyboardInterrupt:
    gpio.cleanup()

# Lab 4 PROGRAM TO SWITCH ON RELAY

In [ ]:
import time
import RPi.GPIO as gpio

gpio.setwarnings(False)
gpio.setmode(gpio.BOARD)

relay1 = 38
gpio.setup(relay1, gpio.OUT, initial=0)

try:
    gpio.output(relay1, True)
    print("relay is switched on ...Please press ctrl+c to exit")
    time.sleep(15)
    gpio.output(relay1, False)
    print("relay is switched off")

except KeyboardInterrupt:
    gpio.cleanup()
    print("program exited")

# Lab 5 PROGRAM TO CAPTURE AN IMAGE USING PICAMERA

In [ ]:
from picamera import PiCamera
from time import sleep
import datetime

camera = PiCamera()
camera.start_preview()
current_date = datetime.datetime.now().strftime('%d-%b-%Y_%H:%M:%S')
sleep(3)
camera.capture('/home/pi/Desktop/MCAsyllabus/images/' + current_date + '.jpg')
camera.stop_preview()
print("Image captured")

# Lab 6 CONTROLLING LIGHT ATUS USING WEB PAGE


In [ ]:
import RPi.GPIO as GPIO
import time
import datetime
from flask import Flask, render_template

led = 13
GPIO.setmode(GPIO.BOARD)
GPIO.setwarnings(False)
GPIO.setup(led, GPIO.OUT, initial=0)

app = Flask(__name__)

@app.route('/')
def hello_world():
    return render_template('Lab6.html')

@app.route('/redledon')
def redledon():
    GPIO.output(led, GPIO.LOW)
    now = datetime.datetime.now()
    timeString = now.strftime("%Y-%m-%d %H:%M:%S")
    templateData = {
        'status' : 'ON',
        'time' : timeString
    }
    return render_template('Lab6.html', **templateData)

@app.route('/redledoff')
def redledoff():
    GPIO.output(led, GPIO.HIGH)
    now = datetime.datetime.now()
    timeString = now.strftime("%Y-%m-%d %H:%M:%S")
    templateData = {
        'status' : 'OFF',
        'time' : timeString
    }
    return render_template('Lab6.html', **templateData)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=80, debug=True)

# Lab 8 READING LIGHT STATUS FROM REMOTE PLACE


In [ ]:
import time
import RPi.GPIO as gpio
from flask import Flask, render_template
import datetime

app = Flask(__name__)

gpio.setwarnings(False)
gpio.setmode(gpio.BOARD)

led1 = 13
switch1 = 35

gpio.setup(led1, gpio.OUT, initial=gpio.LOW)
gpio.setup(switch1, gpio.IN)

light_status = "OFF"
                
def glow_led(event):
    print("Entered Here")
    global light_status
    if event == switch1 and light_status == "OFF":
        gpio.output(led1, True)
        light_status = "ON"
    elif event == switch1 and light_status == "ON":
        gpio.output(led1, False)
        light_status = "OFF"

@app.route('/')
def ledstatus():
    now = datetime.datetime.now()
    timeString = now.strftime("%Y-%m-%d %H:%M:%S")
    templateData = {
        'status' : light_status,
        'time' : timeString
    }
    return render_template('Lab8.html', **templateData)

gpio.add_event_detect(switch1, gpio.RISING, callback=glow_led, bouncetime=100)

if __name__ == '__main__':
    app.run(debug=True, port=2000, host='192.168.35.85')

# Lab 9A server program to read gas values


In [ ]:
import socket
import Adafruit_MCP3008
import Adafruit_GPIO.SPI as SPI
import time

HOST = '192.168.43.141'                    # Standard loopback interface address (localhost)
PORT = 4000                                         


SPI_DEVICE = 0
SPI_PORT = 0

mcp = Adafruit_MCP3008.MCP3008(spi=SPI.SpiDev(SPI_PORT, SPI_DEVICE))

try:
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind((HOST, PORT))
        s.listen()
        conn, addr = s.accept()
        with conn:
            print('Connected by', addr)
            while True:
                value = mcp.read_adc(0)
                print("Gas Value ", value , "units")
                if(value >300):
                    data = "Alert".encode('utf-8')
                    conn.sendall(data)
                time.sleep(3)
except KeyboardInterrupt:
    s.close()
    GPIO.cleanup()

# Lab 9B client program to alert through buzzer


In [ ]:
import socket
import RPi.GPIO as GPIO
import time

Buzzer = 36

HOST = '192.168.43.141'              # The server's hostname or IP address
PORT = 4000                                    

GPIO.setmode(GPIO.BOARD)
GPIO.setup(36, GPIO.OUT)
GPIO.setwarnings(False)

try:
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect((HOST,PORT))
        
        while True:
            data = s.recv(1024).decode('utf-8')
            print(data)
            if(str(data) == 'Alert'):
                print("ALert! Gas Leakage detected")
                GPIO.output(36, True)
                time.sleep(3)
                GPIO.output(36, False)
                time.sleep(3)
except KeyboardInterrupt:
    s.close()
    GPIO.cleanup()